In [1]:
#from google.colab import drive
#drive.mount("/content/drive", force_remount=True)

## Import Dependencies

In [1]:
#file
import json
import xml.etree.ElementTree as ET
import os

#data science
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import numpy as np

#images
from PIL import Image
import cv2
import h5py
from skimage import util
from skimage.measure import label, regionprops
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
# from skimage.measure import compare_psnr updated to peak_signal_noise_ratio
# from skimage.measure import compare_ssim updated to structural_similarity

#stats
import math 
import random
import scipy.io as sio
import scipy
from scipy.ndimage import gaussian_filter, morphology

#torch modules
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import StepLR, MultiStepLR

#model
from sklearn import linear_model

#time
import argparse
from time import time

## Adjust And Format Image Files

In [2]:
def read_image(x):
    img_arr = np.array(Image.open(x))
    if len(img_arr.shape) == 2:  # grayscale
        img_arr = np.tile(img_arr, [3, 1, 1]).transpose(1, 2, 0)
    return img_arr

In [3]:
class RandomCrop(object):
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):

        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        h, w = image.shape[:2]

        if isinstance(self.output_size, tuple):
            new_h = min(self.output_size[0], h)
            new_w = min(self.output_size[1], w)
            assert (new_h, new_w) == self.output_size
        else:
            crop_size = min(self.output_size, h, w)
            assert crop_size == self.output_size
            new_h = new_w = crop_size
        if gtcount > 0:
            mask = target > 0
            ch, cw = int(np.ceil(new_h / 2)), int(np.ceil(new_w / 2))
            mask_center = np.zeros((h, w), dtype=np.uint8)
            mask_center[ch:h-ch+1, cw:w-cw+1] = 1
            mask = (mask & mask_center)
            idh, idw = np.where(mask == 1)
            if len(idh) != 0:
                ids = random.choice(range(len(idh)))
                hc, wc = idh[ids], idw[ids]
                top, left = hc-ch, wc-cw
            else:
                top = np.random.randint(0, h-new_h+1)
                left = np.random.randint(0, w-new_w+1)
        else:
            top = np.random.randint(0, h-new_h+1)
            left = np.random.randint(0, w-new_w+1)

        image = image[top:top+new_h, left:left+new_w, :]
        target = target[top:top+new_h, left:left+new_w]

        return {'image': image, 'target': target, 'gtcount': gtcount}

In [4]:
class RandomFlip(object):
    def __init__(self):
        pass

    def __call__(self, sample):
        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        do_mirror = np.random.randint(2)
        if do_mirror:
            image = cv2.flip(image, 1)
            target = cv2.flip(target, 1)
        return {'image': image, 'target': target, 'gtcount': gtcount}

In [5]:
class Normalize(object):

    def __init__(self, scale, mean, std):
        self.scale = scale
        self.mean = mean
        self.std = std

    def __call__(self, sample):
        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        image, target = image.astype('float32'), target.astype('float32')

        # pixel normalization
        image = (self.scale * image - self.mean) / self.std

        image, target = image.astype('float32'), target.astype('float32')

        return {'image': image, 'target': target, 'gtcount': gtcount}

In [6]:
class ZeroPadding(object):
    def __init__(self, psize=32):
        self.psize = psize

    def __call__(self, sample):
        psize =  self.psize

        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        h,w = image.size()[-2:]
        ph,pw = (psize-h%psize),(psize-w%psize)
        print(ph,pw)

        (pl, pr) = (pw//2, pw-pw//2) if pw != psize else (0, 0)
        (pt, pb) = (ph//2, ph-ph//2) if ph != psize else (0, 0)
        if (ph!=psize) or (pw!=psize):
            tmp_pad = [pl, pr, pt, pb]
            print(tmp_pad)
            image = F.pad(image,tmp_pad)
            target = F.pad(target,tmp_pad)

        return {'image': image, 'target': target, 'gtcount': gtcount}

In [7]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __init__(self):
        pass

    def __call__(self, sample):
        # swap color axis
        # numpy image: H x W x C
        # torch image: C X H X W
        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        image = image.transpose((2, 0, 1))
        target = np.expand_dims(target, axis=2)
        target = target.transpose((2, 0, 1))
        image, target = torch.from_numpy(image), torch.from_numpy(target)
        return {'image': image, 'target': target, 'gtcount': gtcount}

In [8]:
class MaizeTasselDataset(Dataset):
    def __init__(self, data_dir, data_list, ratio, train=True, transform=None):
        self.data_dir = data_dir
        self.data_list = [name.split('\t') for name in open(data_list).read().splitlines()]
        self.ratio = ratio
        self.train = train
        self.transform = transform
        self.image_list = []
        
        # store images and generate ground truths
        self.images = {}
        self.targets = {}
        self.gtcounts = {}
        self.dotimages = {}

    def bbs2points(self, bbs):    
        points = []
        for bb in bbs:
            x1, y1, w, h = [float(b) for b in bb]
            x2, y2 = x1+w-1, y1+h-1
            x, y = np.round((x1+x2)/2).astype(np.int32), np.round((y1+y2)/2).astype(np.int32)
            points.append([x, y])
        return points

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        file_name = self.data_list[idx]
        self.image_list.append(file_name[0])
        if file_name[0] not in self.images:
            image = read_image(self.data_dir+file_name[0])
            annotation = sio.loadmat(self.data_dir+file_name[1])
            h, w = image.shape[:2]
            nh = int(np.ceil(h * self.ratio))
            nw = int(np.ceil(w * self.ratio))
            image = cv2.resize(image, (nw, nh), interpolation = cv2.INTER_CUBIC)
            target = np.zeros((nh, nw), dtype=np.float32)
            dotimage = image.copy()
            if annotation['annotation'][0][0][1] is not None:
                bbs = annotation['annotation'][0][0][1]
                gtcount = bbs.shape[0]
                pts = self.bbs2points(bbs)
                for pt in pts:
                    pt[0], pt[1] = int(pt[0] * self.ratio), int(pt[1] * self.ratio)
                    target[pt[1], pt[0]] = 1
                    cv2.circle(dotimage, (pt[0], pt[1]), int(24 * self.ratio) , (255, 0, 0), -1)
            else:
                gtcount = 0
            target = gaussian_filter(target, 80 * self.ratio)

            #plt.imshow(target, cmap=cm.jet)
            #plt.show()
            print(target.sum())

            self.images.update({file_name[0]:image})
            self.targets.update({file_name[0]:target})
            self.gtcounts.update({file_name[0]:gtcount})
            self.dotimages.update({file_name[0]:dotimage})

        
        sample = {
            'image': self.images[file_name[0]], 
            'target': self.targets[file_name[0]], 
            'gtcount': self.gtcounts[file_name[0]]
        }

        if self.transform:
            sample = self.transform(sample)

        return sample

## Build Data Loader

In [9]:
dataset = MaizeTasselDataset(
        data_dir='maize_counting_dataset', 
        data_list='maize_counting_dataset/train.txt',
        ratio=0.167, 
        train=True, 
        transform=transforms.Compose([
            ToTensor()]
        )
    )

In [10]:
dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=1,
        shuffle=False,
        num_workers=0
    )

In [12]:
print(len(dataloader))
mean = 0.
std = 0.
for i, data in enumerate(dataloader, 0):
        images, targets = data['image'], data['target']
        bs = images.size(0)
        images = images.view(bs, images.size(1), -1).float()
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        print(f"index = {i} size = {images.size()}")
mean /= len(dataloader)
std /= len(dataloader)
print(mean/255.)
print(std/255.)

186
96.999985
index = 0 size = torch.Size([1, 3, 278770])
39.999996
index = 1 size = torch.Size([1, 3, 278770])
22.999998
index = 2 size = torch.Size([1, 3, 278770])
44.0
index = 3 size = torch.Size([1, 3, 278770])
32.0
index = 4 size = torch.Size([1, 3, 278770])
49.000004
index = 5 size = torch.Size([1, 3, 222530])
40.999992
index = 6 size = torch.Size([1, 3, 278770])
60.000004
index = 7 size = torch.Size([1, 3, 222530])
32.000004
index = 8 size = torch.Size([1, 3, 278770])
41.999996
index = 9 size = torch.Size([1, 3, 278770])
66.0
index = 10 size = torch.Size([1, 3, 278770])
65.99999
index = 11 size = torch.Size([1, 3, 278770])
63.0
index = 12 size = torch.Size([1, 3, 278770])
24.0
index = 13 size = torch.Size([1, 3, 278770])
39.999992
index = 14 size = torch.Size([1, 3, 278770])
36.000004
index = 15 size = torch.Size([1, 3, 278770])
42.0
index = 16 size = torch.Size([1, 3, 278770])
49.99999
index = 17 size = torch.Size([1, 3, 278770])
66.0
index = 18 size = torch.Size([1, 3, 278770]

40.000004
index = 155 size = torch.Size([1, 3, 278770])
5.0
index = 156 size = torch.Size([1, 3, 278770])
64.0
index = 157 size = torch.Size([1, 3, 278770])
120.0
index = 158 size = torch.Size([1, 3, 278770])
70.0
index = 159 size = torch.Size([1, 3, 278770])
60.999996
index = 160 size = torch.Size([1, 3, 278770])
24.000002
index = 161 size = torch.Size([1, 3, 222530])
66.00001
index = 162 size = torch.Size([1, 3, 278770])
0.0
index = 163 size = torch.Size([1, 3, 278770])
62.999996
index = 164 size = torch.Size([1, 3, 278770])
37.999992
index = 165 size = torch.Size([1, 3, 278770])
16.000002
index = 166 size = torch.Size([1, 3, 278770])
1.0
index = 167 size = torch.Size([1, 3, 278770])
16.0
index = 168 size = torch.Size([1, 3, 278770])
21.999994
index = 169 size = torch.Size([1, 3, 278770])
0.0
index = 170 size = torch.Size([1, 3, 278770])
40.0
index = 171 size = torch.Size([1, 3, 278770])
0.0
index = 172 size = torch.Size([1, 3, 278770])
14.000001
index = 173 size = torch.Size([1, 3, 

## Computation Functions

In [11]:
def compute_mae(pd, gt):
    pd, gt = np.array(pd), np.array(gt)
    diff = pd - gt
    mae = np.mean(np.abs(diff))
    return mae


def compute_mse(pd, gt):
    pd, gt = np.array(pd), np.array(gt)
    diff = pd - gt
    mse = np.sqrt(np.mean((diff ** 2)))
    return mse

def compute_relerr(pd, gt):
    pd, gt = np.array(pd), np.array(gt)
    diff = pd - gt
    diff = diff[gt > 0]
    gt = gt[gt > 0]
    if (diff is not None) and (gt is not None):
        rmae = np.mean(np.abs(diff) / gt) * 100
        rmse = np.sqrt(np.mean(diff**2 / gt**2)) * 100
    else:
        rmae = 0
        rmse = 0
    return rmae, rmse


def rsquared(pd, gt):
    """ Return R^2 where x and y are array-like."""
    pd, gt = np.array(pd), np.array(gt)
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(pd, gt)
    return r_value**2


def dense_sample2d(x, sx, stride):
    (h,w) = x.shape[:2]
    #idx_img = np.array([i for i in range(h*w)]).reshape(h,w)
    idx_img = np.zeros((h,w),dtype=float)
    
    th = [i for i in range(0, h-sx+1, stride)]
    tw = [j for j in range(0, w-sx+1, stride)]
    norm_vec = np.zeros(len(th)*len(tw))

    for i in th:
        for j in tw:
            idx_img[i:i+sx,j:j+sx] = idx_img[i:i+sx,j:j+sx]+1

    # plot redundancy map
    import os
    import matplotlib.pyplot as plt
    cmap = plt.cm.get_cmap('hot')
    idx_img = idx_img / (idx_img.max())
    idx_img = cmap(idx_img) * 255.
    plt.figure()
    plt.imshow(idx_img.astype(np.uint8))
    plt.axis('off')
    plt.savefig(os.path.join('redundancy_map.pdf'), bbox_inches='tight', dpi = 300)
    plt.close()
   
    idx_img = 1/idx_img
    idx_img = idx_img/sx/sx
    #line order
    idx = 0
    for i in th:
        for j in tw:
            norm_vec[idx] =idx_img[i:i+sx,j:j+sx].sum()
            idx+=1
    
    return norm_vec


def recover_countmap(pred, image, patch_sz, stride):
    pred = pred.reshape(-1)
    imH, imW = image.shape[2:4]
    cntMap = np.zeros((imH, imW), dtype=float)
    norMap = np.zeros((imH, imW), dtype=float)
    
    H = np.arange(0, imH - patch_sz + 1, stride)
    W = np.arange(0, imW - patch_sz + 1, stride)
    cnt = 0
    for h in H:
        for w in W:
            pixel_cnt = pred[cnt] / patch_sz / patch_sz
            cntMap[h:h+patch_sz, w:w+patch_sz] += pixel_cnt
            norMap[h:h+patch_sz, w:w+patch_sz] += np.ones((patch_sz,patch_sz))
            cnt += 1
    return cntMap / (norMap + 1e-12)

## Building the Model

In [12]:
class Encoder(nn.Module):
    def __init__(self, arc='tasselnetv2'):
        super(Encoder, self).__init__()
        if arc == 'tasselnetv2':
            self.encoder = nn.Sequential(
                nn.Conv2d(3, 16, 3, padding=1, bias=False),
                nn.BatchNorm2d(16),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(16, 32, 3, padding=1, bias=False),
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(32, 64, 3, padding=1, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(64, 128, 3, padding=1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 128, 3, padding=1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
            )
        else:
            raise NotImplementedError

    def forward(self, x):
        x = self.encoder(x)
        return x

In [13]:
class Counter(nn.Module):
    def __init__(self, arc='tasselnetv2', input_size=64, output_stride=8):
        super(Counter, self).__init__()
        k = int(input_size / 8)
        avg_pool_stride = int(output_stride / 8)

        if arc == 'tasselnetv2':
            self.counter = nn.Sequential(
                nn.Conv2d(128, 128, (k, k), bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 128, 1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 1, 1)
            )
        else:
            raise NotImplementedError

    def forward(self, x):
        x = self.counter(x)
        return x

In [14]:
class Normalizer:
    @staticmethod
    def cpu_normalizer(x, imh, imw, insz, os):
        # CPU normalization
        bs = x.size()[0]
        normx = np.zeros((imh, imw))
        norm_vec = dense_sample2d(normx, insz, os).astype(np.float32)
        x = x.cpu().detach().numpy().reshape(bs, -1) * norm_vec
        return x
    
    @staticmethod
    def gpu_normalizer(x, imh, imw, insz, os):
        _, _, h, w = x.size()            
        #accm = torch.cuda.FloatTensor(1, insz*insz, h*w).fill_(1)     
        accm = torch.FloatTensor(1, insz*insz, h*w).fill_(1) 
        accm = F.fold(accm, (imh, imw), kernel_size=insz, stride=os)
        accm = 1 / accm
        accm /= insz**2
        accm = F.unfold(accm, kernel_size=insz, stride=os).sum(1).view(1, 1, h, w)
        x *= accm
        return x.squeeze().cpu().detach().numpy()

In [15]:
class CountingModels(nn.Module):
    def __init__(self, arc='tasselnetv2', input_size=64, output_stride=8):
        super(CountingModels, self).__init__()
        self.input_size = input_size
        self.output_stride = output_stride

        self.encoder = Encoder(arc)
        self.counter = Counter(arc, input_size, output_stride)
        if arc == 'tasselnetv2':
          #changed
            self.normalizer = Normalizer.gpu_normalizer
        
        self.weight_init()

    def forward(self, x, is_normalize=True):
        imh, imw = x.size()[2:]
        x = self.encoder(x)
        x = self.counter(x)
        if is_normalize:
            x = self.normalizer(x, imh, imw, self.input_size, self.output_stride)
        return x

    def weight_init(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                # nn.init.kaiming_uniform_(
                #         m.weight, 
                #         mode='fan_in', 
                #         nonlinearity='relu'
                #         )
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

In [16]:
from time import time

insz, os = 64, 8
imH, imW = 1080, 1920
#net = CountingModels(arc='tasselnetv2', input_size=insz, output_stride=os).cuda()
net = CountingModels(arc='tasselnetv2', input_size=insz, output_stride=os)
with torch.no_grad():
    net.eval()
    #x = torch.randn(1, 3, imH, imW).cuda()
    x = torch.randn(1, 3, imH, imW)
    y = net(x)
    print(y.shape)    

with torch.no_grad():
    frame_rate = np.zeros((100, 1))
    for i in range(100):
        #x = torch.randn(1, 3, imH, imW).cuda()
        x = torch.randn(1, 3, imH, imW)
        #torch.cuda.synchronize()
        start = time()

        y = net(x)

        #torch.cuda.synchronize()
        end = time()

        running_frame_rate = 1 * float(1 / (end - start))
        frame_rate[i] = running_frame_rate
    print(np.mean(frame_rate))

(128, 233)
0.7184207871412042


In [17]:
y1 = np.argmax(y, axis=1)
y1

array([231,   2, 232, 232, 232, 231, 118,   0,   0,   2, 175,  43,   0,
       232, 192, 231, 232, 232, 232, 231,   2, 232,   0, 231,   0,   0,
       232,   1, 111,   2,   1, 230, 231, 232, 232, 232,   1, 231, 231,
       209, 231, 232, 230, 232, 232,   2, 167, 232, 232, 206, 232,   0,
        88, 107,   1, 232, 232, 232, 232, 116,   0, 232, 232, 231, 232,
       232, 232, 231, 208, 231, 232,   0, 232,  59, 232, 232, 232,   0,
       232,   1,   0, 232, 232,   0,  42, 231, 232,   1, 232,   1,   0,
         0,   2,   0, 152, 232, 229, 232,   1,   0,   0,   1, 231,   2,
       101,  35, 232, 232, 232, 232,   0, 232, 181,   0,   0,  90, 228,
         0,   2, 232,   0, 231, 232, 231,   1, 231,   0,   1], dtype=int64)

In [18]:
plt.switch_backend('agg')

In [49]:
# prevent dataloader deadlock, uncomment if deadlock occurs
cv2.setNumThreads(0)
#cudnn.enabled = True

# constant
IMG_SCALE = 1. / 255
IMG_MEAN = [.3405, .4747, .2418]
IMG_STD = [1, 1, 1]

# system-related parameters
DATA_DIR = 'maize_counting_dataset'
DATASET = 'mtc'
EXP = 'exp_directory'
DATA_LIST = 'maize_counting_dataset/train.txt'
DATA_VAL_LIST = 'maize_counting_dataset/test.txt'

RESTORE_FROM = 'model_best.pth.tar'
SNAPSHOT_DIR = './snapshots'
RESULT_DIR = './results'

# model-related parameters
INPUT_SIZE = 64
OUTPUT_STRIDE = 8
MODEL = 'tasselnetv2'
RESIZE_RATIO = 0.125

# training-related parameters
OPTIMIZER = 'sgd'  # choice in ['sgd', 'adam']
BATCH_SIZE = 9
CROP_SIZE = (256, 256)
LEARNING_RATE = 1e-2
MILESTONES = [200, 400]
MOMENTUM = 0.95
MULT = 1
NUM_EPOCHS = 50
#NUM_EPOCHS = 500
NUM_CPU_WORKERS = 0
PRINT_EVERY = 1
RANDOM_SEED = 6
WEIGHT_DECAY = 5e-4
VAL_EVERY = 1

device = torch.device("cpu")
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# add a new entry here if creating a new data loader
dataset_list = {
    'mtc': MaizeTasselDataset
}

In [50]:
def save_checkpoint(state, snapshot_dir, filename='model_ckpt.pth.tar'):
    torch.save(state, '{}/{}'.format(snapshot_dir, filename))


def plot_learning_curves(net, dir_to_save):
    # plot learning curves
    fig = plt.figure(figsize=(16, 9))
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.plot(net.train_loss['epoch_loss'], label='train loss', color='tab:blue')
    ax1.legend(loc='upper right')
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.plot(net.val_loss['epoch_loss'], label='val mae', color='tab:orange')
    ax2.legend(loc='upper right')
    # ax2.set_ylim((0,50))
    fig.savefig(os.path.join(dir_to_save, 'learning_curves.png'), bbox_inches='tight', dpi=300)
    plt.close()

## Train and Validate Methods

In [51]:
def train(net, train_loader, criterion, optimizer, epoch, args):
    # switch to 'train' mode
    net.train()

    # uncomment the following line if the training images don't have the same size
    #cudnn.benchmark = True

    if BATCH_SIZE == 1:
        for m in net.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()

    running_loss = 0.0
    avg_frame_rate = 0.0
    in_sz = INPUT_SIZE
    os = OUTPUT_STRIDE
    #target_filter = torch.cuda.FloatTensor(1, 1, in_sz, in_sz).fill_(1)
    target_filter = torch.FloatTensor(1, 1, in_sz, in_sz).fill_(1)
    for i, sample in enumerate(train_loader):
        #torch.cuda.synchronize()
        start = time()

        inputs, targets = sample['image'], sample['target']
        #inputs, targets = inputs.cuda(), targets.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = net(inputs, is_normalize=False)
        # generate targets
        targets = F.conv2d(targets, target_filter, stride=os)
        # compute loss
        loss = criterion(outputs, targets)

        # backward + optimize
        loss.backward()
        optimizer.step()
        # collect and print statistics
        running_loss += loss.item()

        #torch.cuda.synchronize()
        end = time()

        running_frame_rate = BATCH_SIZE * float(1 / (end - start))
        avg_frame_rate = (avg_frame_rate * i + running_frame_rate) / (i + 1)
        if i % PRINT_EVERY == PRINT_EVERY - 1:
            print('epoch: %d, train: %d/%d, '
                  'loss: %.5f, frame: %.2fHz/%.2fHz' % (
                      epoch,
                      i + 1,
                      len(train_loader),
                      running_loss / (i + 1),
                      running_frame_rate,
                      avg_frame_rate
                  ))
    net.train_loss['epoch_loss'].append(running_loss / (i + 1))

In [52]:
def validate(net, valset, val_loader, criterion, epoch, args):
    # switch to 'eval' mode
    net.eval()
    cudnn.benchmark = False

    image_list = valset.image_list

    #if args.save_output:
     #   epoch_result_dir = os.path.join(args.result_dir, str(epoch))
      #  if not os.path.exists(epoch_result_dir):
       #     os.makedirs(epoch_result_dir)
        #cmap = plt.cm.get_cmap('jet')
    epoch_result_dir = os.path.join(RESULT_DIR, str(epoch))
    if not os.path.exists(epoch_result_dir):
        os.makedirs(epoch_result_dir)
    cmap = plt.cm.get_cmap('jet')
    
    pd_counts = []
    gt_counts = []
    with torch.no_grad():
        avg_frame_rate = 0.0
        for i, sample in enumerate(val_loader):
            #torch.cuda.synchronize()
            start = time()

            image, gtcount = sample['image'], sample['gtcount']
            #inference
            #output = net(image.cuda(), is_normalize=not args["save_output"])
            output = net(image, is_normalize=not args["save_output"])
            #if args.save_output:
            #    output_save = output
                # normalization
            #    output = Normalizer.gpu_normalizer(output, image.size()[2], image.size()[3], args.input_size,
            #                                       args.output_stride)
            output_save = output
            output = Normalizer.gpu_normalizer(output, image.size()[2], image.size()[3], INPUT_SIZE,
                                                   OUTPUT_STRIDE)
            
            # postprocessing
            output = np.clip(output, 0, None)

            pdcount = output.sum()
            gtcount = float(gtcount.numpy())

            #if args.save_output:
#             if args["save_output"]:
#                 _, image_name = os.path.split(image_list[i])
#                 output_save = np.clip(output_save.squeeze().cpu().numpy(), 0, None)
#                 output_save = recover_countmap(output_save, image, INPUT_SIZE, OUTPUT_STRIDE)
#                 output_save = output_save / (output_save.max() + 1e-12)
#                 output_save = cmap(output_save) * 255.
#                 # image composition
#                 image = valset.images[image_list[i]]
#                 nh, nw = output_save.shape[:2]
#                 image = cv2.resize(image, (nw, nh), interpolation=cv2.INTER_CUBIC)
#                 output_save = 0.5 * image + 0.5 * output_save[:, :, 0:3]

#                 dotimage = valset.dotimages[image_list[i]]

#                 fig = plt.figure()
#                 ax1 = fig.add_subplot(1, 2, 1)
#                 ax1.imshow(dotimage.astype(np.uint8))
#                 ax1.get_xaxis().set_visible(False)
#                 ax1.get_yaxis().set_visible(False)
#                 ax2 = fig.add_subplot(1, 2, 2)
#                 ax2.imshow(output_save.astype(np.uint8))
#                 ax2.get_xaxis().set_visible(False)
#                 ax2.get_yaxis().set_visible(False)
#                 fig.suptitle('manual count=%4.2f, inferred count=%4.2f' % (gtcount, pdcount), fontsize=10)
#                 if DATASET == 'mtc':
#                     plt.tight_layout(rect=[0, 0, 1, 1.4])  # maize tassels counting
#                 plt.savefig(os.path.join(epoch_result_dir, image_name.replace('.jpg', '.png')), bbox_inches='tight',
#                             dpi=300)
#                 plt.close()

            # compute mae and mse
            pd_counts.append(pdcount)
            gt_counts.append(gtcount)
            mae = compute_mae(pd_counts, gt_counts)
            mse = compute_mse(pd_counts, gt_counts)
            rmae, rmse = compute_relerr(pd_counts, gt_counts)

            #torch.cuda.synchronize()
            end = time()

            running_frame_rate = 1 * float(1 / (end - start))
            avg_frame_rate = (avg_frame_rate * i + running_frame_rate) / (i + 1)
            if i % PRINT_EVERY == PRINT_EVERY - 1:
                print(
                    'epoch: {0}, test: {1}/{2}, pre: {3:.2f}, gt:{4:.2f}, me:{5:.2f}, mae: {6:.2f}, mse: {7:.2f}, rmae: {8:.2f}%, rmse: {9:.2f}%, frame: {10:.2f}Hz/{11:.2f}Hz'
                        .format(epoch, i + 1, len(val_loader), pdcount, gtcount, pdcount - gtcount, mae, mse, rmae,
                                rmse, running_frame_rate, avg_frame_rate)
                )
            start = time()
    r2 = rsquared(pd_counts, gt_counts)
    np.save(SNAPSHOT_DIR + '/pd.npy', pd_counts)
    np.save(SNAPSHOT_DIR + '/gt.npy', gt_counts)
    print('epoch: {0}, mae: {1:.2f}, mse: {2:.2f}, rmae: {3:.2f}%, rmse: {4:.2f}%, r2: {5:.4f}'.format(epoch, mae, mse,
                                                                                                       rmae, rmse, r2))
    # write to files        
    with open(os.path.join(SNAPSHOT_DIR, EXP + '.txt'), 'a') as f:
        print(
            'epoch: {0}, mae: {1:.2f}, mse: {2:.2f}, rmae: {3:.2f}%, rmse: {4:.2f}%, r2: {5:.4f}'.format(epoch, mae,
                                                                                                         mse, rmae,
                                                                                                         rmse, r2),
            file=f
        )
    with open(os.path.join(SNAPSHOT_DIR, 'counts.txt'), 'a') as f:
        for pd, gt in zip(pd_counts, gt_counts):
            print(
                '{0} {1}'.format(pd, gt),
                file=f
            )
    # save stats
    net.val_loss['epoch_loss'].append(mae)
    net.measure['mae'].append(mae)
    net.measure['mse'].append(mse)
    net.measure['rmae'].append(rmae)
    net.measure['rmse'].append(rmse)
    net.measure['r2'].append(r2)

In [53]:
args = {
    "image_scale":IMG_SCALE,
    "image_mean":IMG_MEAN,
    "image_std": IMG_STD,
    "data_dir":DATA_DIR,
    "dataset":DATASET,
    "exp":EXP,
    "data_list":DATA_LIST,
    "data_val_list":DATA_VAL_LIST,
    "restore_from":RESTORE_FROM,
    "snapshot_dir":SNAPSHOT_DIR,
    "result_dir":RESULT_DIR,
    "save_output":"store_true",
    "input_size":INPUT_SIZE,
    "output_stride":OUTPUT_STRIDE,
    "resize_ratio":RESIZE_RATIO,
    "model":MODEL,
    "optimizer":OPTIMIZER,
    "batch_size":BATCH_SIZE,
    "milestones":MILESTONES,
    "crop_size":CROP_SIZE,
    "evaluate_only":"store_true",
    "learning_rate":LEARNING_RATE,
    "momentum":MOMENTUM,
    "weight_decay":WEIGHT_DECAY,
    "mult":MULT,
    "num_epochs":NUM_EPOCHS,
    "num_workers":NUM_CPU_WORKERS,
    "print_every":PRINT_EVERY,
    "random_seed":RANDOM_SEED,
    "val_every":VAL_EVERY
}

In [54]:
import os
IMG_MEAN = np.array(IMG_MEAN).reshape((1, 1, 3))
IMG_STD = np.array(IMG_STD).reshape((1, 1, 3))

CROP_SIZE = tuple(CROP_SIZE) if len(CROP_SIZE) > 1 else CROP_SIZE

# seeding for reproducbility
#if torch.cuda.is_available():
#    torch.cuda.manual_seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# instantiate dataset
dataset = dataset_list[DATASET]

SNAPSHOT_DIR = os.path.join(SNAPSHOT_DIR, DATASET.lower(), EXP)
if not os.path.exists(SNAPSHOT_DIR):
    os.makedirs(SNAPSHOT_DIR)

RESULT_DIR = os.path.join(RESULT_DIR, DATASET.lower(), EXP)
if not os.path.exists(RESULT_DIR):
    os.makedirs(RESULT_DIR)

RESTORE_FROM = os.path.join(SNAPSHOT_DIR, RESTORE_FROM)

#arguments = vars(args)
#for item in arguments:
    #print(item, ':\t', arguments[item])

In [55]:
# instantiate network
net = CountingModels(
      arc=MODEL,
      input_size=INPUT_SIZE,
      output_stride=OUTPUT_STRIDE
    )

net = nn.DataParallel(net)
#net.cuda()

# filter parameters
learning_params = [p[1] for p in net.named_parameters()]
pretrained_params = []

# define loss function and optimizer
#criterion = nn.L1Loss(reduction='mean').cuda()
criterion = nn.L1Loss(reduction='mean')

if OPTIMIZER == 'sgd':
    optimizer = torch.optim.SGD(
        [
            {'params': learning_params},
            {'params': pretrained_params, 'lr': LEARNING_RATE / MULT},
        ],
        lr=LEARNING_RATE,
        momentum=MOMENTUM,
        weight_decay=WEIGHT_DECAY
    )
elif OPTIMIZER == 'adam':
    optimizer = torch.optim.Adam(
        [
            {'params': learning_params},
            {'params': pretrained_params, 'lr': LEARNING_RATE / MULT},
        ],
        lr=LEARNING_RATE
    )
else:
    raise NotImplementedError

In [56]:
# restore parameters
start_epoch = 0
net.train_loss = {
    'running_loss': [],
    'epoch_loss': []
}
net.val_loss = {
    'running_loss': [],
    'epoch_loss': []
}
net.measure = {
    'mae': [],
    'mse': [],
    'rmae': [],
    'rmse': [],
    'r2': []
}
if RESTORE_FROM is not None:
    if os.path.isfile(RESTORE_FROM):
        checkpoint = torch.load(RESTORE_FROM)
        net.load_state_dict(checkpoint['state_dict'])
        if 'epoch' in checkpoint:
            start_epoch = checkpoint['epoch']
        if 'optimizer' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer'])
        if 'train_loss' in checkpoint:
            net.train_loss = checkpoint['train_loss']
        if 'val_loss' in checkpoint:
            net.val_loss = checkpoint['val_loss']
        if 'measure' in checkpoint:
            net.measure['mae'] = checkpoint['measure']['mae'] if 'mae' in checkpoint['measure'] else []
            net.measure['mse'] = checkpoint['measure']['mse'] if 'mse' in checkpoint['measure'] else []
            net.measure['rmae'] = checkpoint['measure']['rmae'] if 'rmae' in checkpoint['measure'] else []
            net.measure['rmse'] = checkpoint['measure']['rmse'] if 'rmse' in checkpoint['measure'] else []
            net.measure['r2'] = checkpoint['measure']['r2'] if 'r2' in checkpoint['measure'] else []
        print("==> load checkpoint '{}' (epoch {})"
              .format(RESTORE_FROM, start_epoch))
    else:
        with open(os.path.join(SNAPSHOT_DIR, EXP + '.txt'), 'a') as f:
            for item in args:
                print(item, ':\t', args[item], file=f)
        print("==> no checkpoint found at '{}'".format(RESTORE_FROM))

==> no checkpoint found at './snapshots\mtc\exp_directory\model_best.pth.tar'


In [57]:
# define transform
transform_train = [
    RandomCrop(CROP_SIZE),
    RandomFlip(),
    Normalize(
        IMG_SCALE,
        IMG_MEAN,
        IMG_STD
    ),
    ToTensor(),
    ZeroPadding(OUTPUT_STRIDE)
]
transform_val = [
    Normalize(
        IMG_SCALE,
        IMG_MEAN,
        IMG_STD
    ),
    ToTensor(),
    ZeroPadding(OUTPUT_STRIDE)
]
composed_transform_train = transforms.Compose(transform_train)
composed_transform_val = transforms.Compose(transform_val)

# define dataset loader
trainset = dataset(
    data_dir=DATA_DIR,
    data_list=DATA_LIST,
    ratio=RESIZE_RATIO,
    train=True,
    transform=composed_transform_train
)
train_loader = DataLoader(
    trainset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_CPU_WORKERS,
    pin_memory=True,
    drop_last=True
)
valset = dataset(
    data_dir=DATA_DIR,
    data_list=DATA_LIST,
    ratio=RESIZE_RATIO,
    train=False,
    transform=composed_transform_val
)
val_loader = DataLoader(
    valset,
    batch_size=1,
    shuffle=False,
    num_workers=NUM_CPU_WORKERS,
    pin_memory=True
)

## Testing Pytorch Deployment Codes

In [28]:
model = torch.load('snapshots/mtc/exp_directory/whole_model.pt')
model.eval()

DataParallel(
  (module): CountingModels(
    (encoder): Encoder(
      (encoder): Sequential(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
        (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (6): ReLU(inplace=True)
        (7): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
        (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (10): ReLU(inplace=True)
        (11): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilati

In [40]:
image = read_image('Sample_Images/T0006_XM_20120815160306_01.jpg')
image.shape

(2736, 3648, 3)

In [43]:
#Resize
#img_file = Image.open('Sample_Images/T0006_XM_20120815160306_01.jpg')
image = read_image('Sample_Images/T0006_XM_20120815160306_01.jpg')
annotation = sio.loadmat('Sample_Images/T0006_XM_20120815160306_01.mat')
h, w = image.shape[:2]
nh = int(np.ceil(h * 0.125))
nw = int(np.ceil(w * 0.125))
image = cv2.resize(image, (nw, nh), interpolation = cv2.INTER_CUBIC)
#target = np.zeros((nh, nw), dtype=np.float32)
#dotimage = image.copy()
test_img = image.copy()
image.shape

(342, 456, 3)

In [44]:
#Normalize
image = image.astype('float32')
# pixel normalization
image = (IMG_SCALE * image - IMG_MEAN) / IMG_STD
image = image.astype('float32')

In [45]:
image = image.transpose((2, 0, 1))
image = torch.from_numpy(image)

In [46]:
image.size()

torch.Size([3, 342, 456])

In [47]:
#Zero Padding
psize=32
h,w = image.size()[-2:]
ph,pw = (psize-h%psize),(psize-w%psize)

(pl, pr) = (pw//2, pw-pw//2) if pw != psize else (0, 0)
(pt, pb) = (ph//2, ph-ph//2) if ph != psize else (0, 0)
if (ph!=psize) or (pw!=psize):
    tmp_pad = [pl, pr, pt, pb]
    image = F.pad(image,tmp_pad)

In [49]:
image=image.unsqueeze(0)
image.size()

torch.Size([1, 3, 352, 480])

In [56]:
import io
model.eval()
with torch.no_grad():
    output = model(image, is_normalize=not args["save_output"])
    output_save = output
    output = Normalizer.gpu_normalizer(output, image.size()[2], image.size()[3], INPUT_SIZE,OUTPUT_STRIDE)       
    #postprocessing
    output = np.clip(output, 0, None)
    pdcount = output.sum()
    print(pdcount)
    
    #try density map
    cmap = plt.cm.get_cmap('jet')
    output_save = np.clip(output_save.squeeze().cpu().numpy(), 0, None)
    output_save = recover_countmap(output_save, image, INPUT_SIZE, OUTPUT_STRIDE)
    output_save = output_save / (output_save.max() + 1e-12)
    output_save = cmap(output_save) * 255.
    # image composition
    print(output_save.shape)
    test_img = read_image('Sample_Images/T0006_XM_20120815160306_01.jpg')
    nh, nw = output_save.shape[:2]
    test_img = cv2.resize(test_img, (nw, nh), interpolation=cv2.INTER_CUBIC)
    #output_save = test_img + output_save[:, :, 0:3]
    output_save = 0.5 * test_img + 0.5 * output_save[:, :, 0:3]
    
    fig, ax = plt.subplots(figsize=(15,5))
    ax.imshow(output_save.astype(np.uint8))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    #fig = plt.figure()
    #ax1 = fig.add_subplot(1, 2, 1)
    #ax1.imshow(dotimage.astype(np.uint8))
    #ax1.get_xaxis().set_visible(False)
    #ax1.get_yaxis().set_visible(False)
    #ax2 = fig.add_subplot(1, 2, 2)
    #ax2.imshow(output_save.astype(np.uint8))
    #ax2.get_xaxis().set_visible(False)
    #ax2.get_yaxis().set_visible(False)
    
    #fig.suptitle('inferred count=%4.2f' % (pdcount), fontsize=10)
    
    #plt.tight_layout(rect=[0, 0, 1, 1.4])  # maize tassels counting 
    
    #plt.savefig("test.jpg",bbox_inches='tight',dpi=300)
    
    my_stringIObytes = io.BytesIO()
    print(my_stringIObytes)
    plt.savefig(my_stringIObytes, format='jpg',bbox_inches='tight',dpi=72)
    my_stringIObytes.seek(0)
    my_base64_jpgData = base64.b64encode(my_stringIObytes.read())
    print(my_base64_jpgData)
    
    #plt.show()
    #plt.savefig(os.path.join(epoch_result_dir, image_name.replace('.jpg', '.png')), bbox_inches='tight',dpi=300)

21.263014
(352, 480, 4)
b'...'


In [57]:
valset.images

{}

In [ ]:
image = valset.images[image_list[i]]
nh, nw = output_save.shape[:2]
image = cv2.resize(image, (nw, nh), interpolation=cv2.INTER_CUBIC)

In [84]:
with torch.no_grad():
        for i, sample in enumerate(val_loader):
            image123, gtcount = sample['image'], sample['gtcount']
            print(image123.size())

2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
8 8
torch.Size([1, 3, 288, 432])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
8 8
torch.Size([1, 3, 288, 432])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 

In [83]:
image123.size()

torch.Size([1, 3, 344, 456])

## Training the Model

In [58]:
print('alchemy start...')
#if evaluate_only:
#validate(net, valset, val_loader, criterion, start_epoch, args)
    #return

best_mae = 1000000.0
resume_epoch = -1 if start_epoch == 0 else start_epoch
scheduler = MultiStepLR(optimizer, milestones=MILESTONES, gamma=0.1, last_epoch=resume_epoch)
for epoch in range(start_epoch, NUM_EPOCHS):
    # train
    train(net, train_loader, criterion, optimizer, epoch + 1, args)
    if epoch % VAL_EVERY == VAL_EVERY - 1:
        # val
        validate(net, valset, val_loader, criterion, epoch + 1, args)
        # save_checkpoint
        state = {
            'state_dict': net.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch + 1,
            'train_loss': net.train_loss,
            'val_loss': net.val_loss,
            'measure': net.measure
        }
        save_checkpoint(state, SNAPSHOT_DIR, filename='model_ckpt.pth.tar')
        if net.measure['mae'][-1] <= best_mae:
            save_checkpoint(state, SNAPSHOT_DIR, filename='model_best.pth.tar')
            save_checkpoint(state, SNAPSHOT_DIR, filename='model.pth')
            best_mae = net.measure['mae'][-1]
            best_mse = net.measure['mse'][-1]
            best_rmae = net.measure['rmae'][-1]
            best_rmse = net.measure['rmse'][-1]
            best_r2 = net.measure['r2'][-1]
        print(EXP + ' epoch {} finished!'.format(epoch + 1))
        print('best mae: {0:.2f}, best mse: {1:.2f}, best_rmae: {2:.2f}, best_rmse: {3:.2f}, best_r2: {4:.4f}'
              .format(best_mae, best_mse, best_rmae, best_rmse, best_r2))
        plot_learning_curves(net, SNAPSHOT_DIR)
    scheduler.step()

print('Experiments with ' + EXP + ' done!')
with open(os.path.join(SNAPSHOT_DIR, EXP + '.txt'), 'a') as f:
    print(
        'best mae: {0:.2f}, best mse: {1:.2f}, best_rmae: {2:.2f}, best_rmse: {3:.2f}, best_r2: {4:.4f}'
            .format(best_mae, best_mse, best_rmae, best_rmse, best_r2),
        file=f
    )
    print(
        'overall best mae: {0:.2f}, overall best mse: {1:.2f}, overall best_rmae: {2:.2f}, overall best_rmse: {3:.2f}, overall best_r2: {4:.4f}'
            .format(min(net.measure['mae']), min(net.measure['mse']), min(net.measure['rmae']),
                    min(net.measure['rmse']), max(net.measure['r2'])),
        file=f
    )

alchemy start...
57.0
8 8
16.0
8 8
23.0
8 8
49.0
8 8
18.0
8 8
32.0
8 8
65.00001
8 8
14.000002
8 8
67.0
8 8
epoch: 1, train: 1/20, loss: 0.94054, frame: 14.67Hz/14.67Hz
42.000008
8 8
1.0000001
8 8
9.0
8 8
51.000004
8 8
65.99999
8 8
104.00001
8 8
37.000004
8 8
64.00001
8 8
0.0
8 8
epoch: 1, train: 2/20, loss: 0.91269, frame: 16.77Hz/15.72Hz
18.000002
8 8
44.0
8 8
2.0
8 8
85.00001
8 8
17.000002
8 8
38.0
8 8
56.000004
8 8
62.999996
8 8
63.999992
8 8
epoch: 1, train: 3/20, loss: 0.87621, frame: 18.45Hz/16.63Hz
39.999992
8 8
120.0
8 8
69.0
8 8
27.999998
8 8
54.0
8 8
36.000004
8 8
15.999999
8 8
51.0
8 8
19.0
8 8
epoch: 1, train: 4/20, loss: 0.84678, frame: 18.01Hz/16.97Hz
0.0
8 8
68.00001
8 8
38.0
8 8
62.999996
8 8
18.0
8 8
65.99999
8 8
60.0
8 8
70.0
8 8
0.0
8 8
epoch: 1, train: 5/20, loss: 0.83253, frame: 17.52Hz/17.08Hz
62.0
8 8
69.0
8 8
0.0
8 8
30.999998
8 8
41.999996
8 8
1.0
8 8
50.999996
8 8
5.0
8 8
2.9999998
8 8
epoch: 1, train: 6/20, loss: 0.84698, frame: 15.80Hz/16.87Hz
66.0
8 8
54.0


17.000002
2 8
[0, 0, 1, 1]
epoch: 1, test: 33/186, pre: 12.67, gt:17.00, me:-4.33, mae: 32.06, mse: 38.95, rmae: 64.50%, rmse: 67.50%, frame: 11.00Hz/11.21Hz
9.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 34/186, pre: 12.72, gt:9.00, me:3.72, mae: 31.23, mse: 38.38, rmae: 63.78%, rmse: 66.83%, frame: 10.76Hz/11.20Hz
30.999998
2 8
[0, 0, 1, 1]
epoch: 1, test: 35/186, pre: 12.60, gt:31.00, me:-18.40, mae: 30.86, mse: 37.95, rmae: 63.64%, rmse: 66.62%, frame: 10.76Hz/11.19Hz
5.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 36/186, pre: 12.71, gt:5.00, me:7.71, mae: 30.22, mse: 37.44, rmae: 66.31%, rmse: 70.76%, frame: 11.00Hz/11.18Hz
39.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 37/186, pre: 12.93, gt:39.00, me:-26.07, mae: 30.10, mse: 37.18, rmae: 66.32%, rmse: 70.65%, frame: 10.21Hz/11.15Hz
94.00001
2 8
[0, 0, 1, 1]
epoch: 1, test: 38/186, pre: 13.03, gt:94.00, me:-80.97, mae: 31.44, mse: 38.97, rmae: 66.88%, rmse: 71.13%, frame: 10.42Hz/11.14Hz
37.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 39/186, pre: 12.63, gt:37.00, me:

51.999992
2 8
[0, 0, 1, 1]
epoch: 1, test: 86/186, pre: 12.71, gt:52.00, me:-39.29, mae: 31.55, mse: 38.61, rmae: 81.48%, rmse: 149.15%, frame: 11.24Hz/11.27Hz
63.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 87/186, pre: 12.46, gt:63.00, me:-50.54, mae: 31.77, mse: 38.77, rmae: 81.46%, rmse: 148.49%, frame: 10.32Hz/11.26Hz
116.99999
2 8
[0, 0, 1, 1]
epoch: 1, test: 88/186, pre: 12.77, gt:117.00, me:-104.23, mae: 32.59, mse: 40.12, rmae: 81.56%, rmse: 147.90%, frame: 11.63Hz/11.26Hz
0.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 89/186, pre: 12.76, gt:0.00, me:12.76, mae: 32.37, mse: 39.91, rmae: 81.56%, rmse: 147.90%, frame: 11.50Hz/11.26Hz
0.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 90/186, pre: 12.83, gt:0.00, me:12.83, mae: 32.15, mse: 39.71, rmae: 81.56%, rmse: 147.90%, frame: 11.12Hz/11.26Hz
62.999996
2 8
[0, 0, 1, 1]
epoch: 1, test: 91/186, pre: 12.69, gt:63.00, me:-50.31, mae: 32.35, mse: 39.85, rmae: 81.54%, rmse: 147.26%, frame: 11.50Hz/11.26Hz
2.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 92/186, pre: 12.82, gt

44.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 139/186, pre: 12.65, gt:44.00, me:-31.35, mae: 30.69, mse: 37.86, rmae: 103.93%, rmse: 206.89%, frame: 10.32Hz/11.22Hz
62.000004
2 8
[0, 0, 1, 1]
epoch: 1, test: 140/186, pre: 12.47, gt:62.00, me:-49.53, mae: 30.83, mse: 37.96, rmae: 103.75%, rmse: 206.21%, frame: 10.64Hz/11.22Hz
50.999996
2 8
[0, 0, 1, 1]
epoch: 1, test: 141/186, pre: 12.54, gt:51.00, me:-38.46, mae: 30.88, mse: 37.96, rmae: 103.53%, rmse: 205.53%, frame: 10.76Hz/11.21Hz
96.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 142/186, pre: 12.95, gt:96.00, me:-83.05, mae: 31.25, mse: 38.46, rmae: 103.40%, rmse: 204.89%, frame: 11.00Hz/11.21Hz
17.999998
2 8
[0, 0, 1, 1]
epoch: 1, test: 143/186, pre: 12.71, gt:18.00, me:-5.29, mae: 31.07, mse: 38.33, rmae: 102.84%, rmse: 204.13%, frame: 10.88Hz/11.21Hz
99.99999
2 8
[0, 0, 1, 1]
epoch: 1, test: 144/186, pre: 12.99, gt:100.00, me:-87.01, mae: 31.46, mse: 38.88, rmae: 102.73%, rmse: 203.51%, frame: 10.53Hz/11.20Hz
63.999996
2 8
[0, 0, 1, 1]
epoch: 1, te

epoch: 2, train: 6/20, loss: 0.72387, frame: 17.35Hz/17.43Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 2, train: 7/20, loss: 0.68245, frame: 17.62Hz/17.45Hz
63.0
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 2, train: 8/20, loss: 0.65006, frame: 15.50Hz/17.21Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 2, train: 9/20, loss: 0.63112, frame: 17.97Hz/17.29Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 2, train: 10/20, loss: 0.62277, frame: 17.52Hz/17.32Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 2, train: 11/20, loss: 0.61112, frame: 17.15Hz/17.30Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 2, train: 12/20, loss: 0.61533, frame: 17.42Hz/17.31Hz
70.0
8 8
8 8
8 8
8 8
8 8
8 8
30.999996
8 8
8 8
8 8
epoch: 2, train: 13/20, loss: 0.61207, frame: 18.23Hz/17.38Hz
8 8
8 8
8 8
39.0
8 8
8 8
8 8
33.999996
8 8
8 8
8 8
epoch: 2, train: 14/20, loss: 0.60312, frame: 17.83Hz/17.41Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 2, train: 15/20, loss: 0.60802, frame: 18.49Hz/17.49Hz
8 8
8 8
8 8
8 8
8 8
8 8
8

...

2 8
[0, 0, 1, 1]
epoch: 49, test: 163/186, pre: 58.36, gt:66.00, me:-7.64, mae: 10.47, mse: 12.81, rmae: 29.10%, rmse: 43.71%, frame: 11.24Hz/11.20Hz
2 8
[0, 0, 1, 1]
epoch: 49, test: 164/186, pre: 0.09, gt:0.00, me:0.09, mae: 10.41, mse: 12.77, rmae: 29.10%, rmse: 43.71%, frame: 11.12Hz/11.20Hz
2 8
[0, 0, 1, 1]
epoch: 49, test: 165/186, pre: 51.55, gt:63.00, me:-11.45, mae: 10.42, mse: 12.76, rmae: 29.03%, rmse: 43.59%, frame: 11.12Hz/11.20Hz
2 8
[0, 0, 1, 1]
epoch: 49, test: 166/186, pre: 37.62, gt:38.00, me:-0.38, mae: 10.36, mse: 12.72, rmae: 28.85%, rmse: 43.45%, frame: 11.37Hz/11.20Hz
2 8
[0, 0, 1, 1]
epoch: 49, test: 167/186, pre: 19.02, gt:16.00, me:3.02, mae: 10.31, mse: 12.69, rmae: 28.78%, rmse: 43.33%, frame: 11.50Hz/11.20Hz
2 8
[0, 0, 1, 1]
epoch: 49, test: 168/186, pre: 0.23, gt:1.00, me:-0.77, mae: 10.26, mse: 12.65, rmae: 29.10%, rmse: 43.63%, frame: 11.50Hz/11.20Hz
2 8
[0, 0, 1, 1]
epoch: 49, test: 169/186, pre: 14.59, gt:16.00, me:-1.41, mae: 10.20, mse: 12.61, rmae: 

epoch: 50, test: 18/186, pre: 35.58, gt:50.00, me:-14.42, mae: 17.09, mse: 18.95, rmae: 37.10%, rmse: 39.63%, frame: 11.50Hz/11.26Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 19/186, pre: 54.19, gt:66.00, me:-11.81, mae: 16.81, mse: 18.64, rmae: 36.09%, rmse: 38.79%, frame: 11.50Hz/11.28Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 20/186, pre: 34.90, gt:51.00, me:-16.10, mae: 16.78, mse: 18.52, rmae: 35.86%, rmse: 38.46%, frame: 11.37Hz/11.28Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 21/186, pre: 34.56, gt:51.00, me:-16.44, mae: 16.76, mse: 18.43, rmae: 35.69%, rmse: 38.19%, frame: 11.50Hz/11.29Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 22/186, pre: 19.10, gt:31.00, me:-11.90, mae: 16.54, mse: 18.18, rmae: 35.81%, rmse: 38.20%, frame: 10.88Hz/11.27Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 23/186, pre: 81.89, gt:102.00, me:-20.11, mae: 16.70, mse: 18.27, rmae: 35.11%, rmse: 37.58%, frame: 10.64Hz/11.25Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 24/186, pre: 0.01, gt:0.00, me:0.01, mae: 16.00, mse: 17.89, rmae: 35.11%, rmse: 3

2 8
[0, 0, 1, 1]
epoch: 50, test: 74/186, pre: 24.32, gt:36.00, me:-11.68, mae: 14.04, mse: 16.27, rmae: 39.05%, rmse: 51.33%, frame: 10.76Hz/11.23Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 75/186, pre: 19.93, gt:35.00, me:-15.07, mae: 14.05, mse: 16.26, rmae: 39.10%, rmse: 51.22%, frame: 10.88Hz/11.23Hz
8 8
epoch: 50, test: 76/186, pre: 11.96, gt:18.00, me:-6.04, mae: 13.95, mse: 16.17, rmae: 39.02%, rmse: 51.01%, frame: 12.99Hz/11.25Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 77/186, pre: 2.24, gt:14.00, me:-11.76, mae: 13.92, mse: 16.12, rmae: 39.67%, rmse: 51.63%, frame: 10.21Hz/11.24Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 78/186, pre: 10.56, gt:13.00, me:-2.44, mae: 13.77, mse: 16.02, rmae: 39.37%, rmse: 51.31%, frame: 11.24Hz/11.24Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 79/186, pre: 82.43, gt:96.00, me:-13.57, mae: 13.77, mse: 15.99, rmae: 39.02%, rmse: 50.98%, frame: 10.53Hz/11.23Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 80/186, pre: 4.33, gt:9.00, me:-4.67, mae: 13.66, mse: 15.90, rmae: 39.20%, rmse: 

epoch: 50, test: 129/186, pre: 45.53, gt:64.00, me:-18.47, mae: 13.41, mse: 15.86, rmae: 38.87%, rmse: 48.15%, frame: 9.81Hz/11.22Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 130/186, pre: 0.97, gt:3.00, me:-2.03, mae: 13.32, mse: 15.80, rmae: 39.11%, rmse: 48.35%, frame: 11.12Hz/11.22Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 131/186, pre: 7.46, gt:4.00, me:3.46, mae: 13.25, mse: 15.74, rmae: 39.50%, rmse: 48.79%, frame: 11.50Hz/11.22Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 132/186, pre: 37.25, gt:70.00, me:-32.75, mae: 13.40, mse: 15.94, rmae: 39.56%, rmse: 48.77%, frame: 11.37Hz/11.22Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 133/186, pre: 24.02, gt:38.00, me:-13.98, mae: 13.40, mse: 15.93, rmae: 39.54%, rmse: 48.68%, frame: 11.12Hz/11.22Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 134/186, pre: 0.15, gt:1.00, me:-0.85, mae: 13.31, mse: 15.87, rmae: 39.91%, rmse: 49.09%, frame: 10.53Hz/11.22Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 135/186, pre: 32.63, gt:45.00, me:-12.37, mae: 13.30, mse: 15.85, rmae: 39.81%, rmse: 48

2 8
[0, 0, 1, 1]
epoch: 50, test: 184/186, pre: 19.97, gt:33.00, me:-13.03, mae: 13.19, mse: 15.87, rmae: 37.71%, rmse: 45.41%, frame: 11.37Hz/11.20Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 185/186, pre: 75.84, gt:90.00, me:-14.16, mae: 13.20, mse: 15.86, rmae: 37.58%, rmse: 45.29%, frame: 10.88Hz/11.20Hz
2 8
[0, 0, 1, 1]
epoch: 50, test: 186/186, pre: 0.01, gt:0.00, me:0.01, mae: 13.13, mse: 15.82, rmae: 37.58%, rmse: 45.29%, frame: 11.24Hz/11.20Hz
epoch: 50, mae: 13.13, mse: 15.82, rmae: 37.58%, rmse: 45.29%, r2: 0.9349
exp_directory epoch 50 finished!
best mae: 5.03, best mse: 7.23, best_rmae: 21.64, best_rmse: 56.88, best_r2: 0.9345
Experiments with exp_directory done!


In [31]:
torch.save(net, 'snapshots/mtc/exp_directory/whole_model.pt')

## Validate the Model

In [32]:
validate(net, valset, val_loader, criterion, start_epoch, args)

2 8
[0, 0, 1, 1]
epoch: 3, test: 1/186, pre: 118.44, gt:97.00, me:21.44, mae: 21.44, mse: 21.44, rmae: 22.10%, rmse: 22.10%, frame: 1.00Hz/1.00Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 2/186, pre: 38.28, gt:40.00, me:-1.72, mae: 11.58, mse: 15.21, rmae: 13.20%, rmse: 15.92%, frame: 1.03Hz/1.01Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 3/186, pre: 25.41, gt:23.00, me:2.41, mae: 8.52, mse: 12.49, rmae: 12.30%, rmse: 14.34%, frame: 1.01Hz/1.01Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 4/186, pre: 57.43, gt:44.00, me:13.43, mae: 9.75, mse: 12.73, rmae: 16.85%, rmse: 19.68%, frame: 1.03Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 5/186, pre: 29.10, gt:32.00, me:-2.90, mae: 8.38, mse: 11.46, rmae: 15.29%, rmse: 18.06%, frame: 1.02Hz/1.02Hz
8 8
epoch: 3, test: 6/186, pre: 6.51, gt:49.00, me:-42.49, mae: 14.07, mse: 20.26, rmae: 27.20%, rmse: 39.05%, frame: 1.13Hz/1.04Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 7/186, pre: 52.86, gt:41.00, me:11.86, mae: 13.75, mse: 19.29, rmae: 27.45%, rmse: 37.77%, frame: 1.05Hz/1.04Hz
8

epoch: 3, test: 58/186, pre: 78.10, gt:65.00, me:13.10, mae: 10.41, mse: 14.14, rmae: 40.29%, rmse: 102.71%, frame: 1.04Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 59/186, pre: 25.48, gt:21.00, me:4.48, mae: 10.31, mse: 14.03, rmae: 39.94%, rmse: 101.78%, frame: 1.00Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 60/186, pre: 63.69, gt:52.00, me:11.69, mae: 10.34, mse: 13.99, rmae: 39.61%, rmse: 100.88%, frame: 1.02Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 61/186, pre: 51.28, gt:51.00, me:0.28, mae: 10.17, mse: 13.88, rmae: 38.90%, rmse: 99.96%, frame: 1.04Hz/1.02Hz
8 8
epoch: 3, test: 62/186, pre: 31.81, gt:36.00, me:-4.19, mae: 10.08, mse: 13.78, rmae: 38.42%, rmse: 99.08%, frame: 1.21Hz/1.03Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 63/186, pre: 1.53, gt:0.00, me:1.53, mae: 9.94, mse: 13.67, rmae: 38.42%, rmse: 99.08%, frame: 1.05Hz/1.03Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 64/186, pre: 1.66, gt:1.00, me:0.66, mae: 9.79, mse: 13.56, rmae: 38.89%, rmse: 98.59%, frame: 1.00Hz/1.03Hz
2 8
[0, 0, 1, 1

epoch: 3, test: 115/186, pre: 88.08, gt:56.00, me:32.08, mae: 10.13, mse: 13.58, rmae: 33.31%, rmse: 75.62%, frame: 1.03Hz/1.03Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 116/186, pre: 14.37, gt:15.00, me:-0.63, mae: 10.05, mse: 13.52, rmae: 33.03%, rmse: 75.26%, frame: 0.91Hz/1.03Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 117/186, pre: 65.72, gt:52.00, me:13.72, mae: 10.08, mse: 13.52, rmae: 32.97%, rmse: 74.95%, frame: 1.04Hz/1.03Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 118/186, pre: 79.73, gt:47.00, me:32.73, mae: 10.27, mse: 13.80, rmae: 33.31%, rmse: 74.90%, frame: 1.02Hz/1.03Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 119/186, pre: 52.68, gt:51.00, me:1.68, mae: 10.20, mse: 13.74, rmae: 33.03%, rmse: 74.56%, frame: 1.01Hz/1.03Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 120/186, pre: 41.29, gt:41.00, me:0.29, mae: 10.12, mse: 13.69, rmae: 32.74%, rmse: 74.22%, frame: 1.02Hz/1.03Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 121/186, pre: 50.05, gt:45.00, me:5.05, mae: 10.07, mse: 13.64, rmae: 32.55%, rmse: 73.89%, frame: 1.01H

epoch: 3, test: 172/186, pre: 73.31, gt:40.00, me:33.31, mae: 10.50, mse: 14.17, rmae: 34.92%, rmse: 74.40%, frame: 1.03Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 173/186, pre: 0.96, gt:0.00, me:0.96, mae: 10.45, mse: 14.13, rmae: 34.92%, rmse: 74.40%, frame: 0.99Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 174/186, pre: 10.47, gt:14.00, me:-3.53, mae: 10.41, mse: 14.09, rmae: 34.86%, rmse: 74.20%, frame: 1.02Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 175/186, pre: 57.51, gt:54.00, me:3.51, mae: 10.37, mse: 14.06, rmae: 34.68%, rmse: 73.97%, frame: 1.01Hz/1.02Hz
8 8
epoch: 3, test: 176/186, pre: 45.53, gt:64.00, me:-18.47, mae: 10.42, mse: 14.08, rmae: 34.65%, rmse: 73.77%, frame: 1.11Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 177/186, pre: 37.79, gt:41.00, me:-3.21, mae: 10.38, mse: 14.05, rmae: 34.48%, rmse: 73.55%, frame: 1.00Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 178/186, pre: 48.05, gt:46.00, me:2.05, mae: 10.33, mse: 14.01, rmae: 34.30%, rmse: 73.33%, frame: 1.01Hz/1.02Hz
2 8
[